## A notebook to seamlessly take blast output to GO Slim list

This is a notebook meant to run in a working directory. Please set working directory as variable in next cell

In [10]:
cd /Users/sr320/Documents/GitHub/nb-2018/O_lurida

/Users/sr320/Documents/GitHub/nb-2018/O_lurida


In [11]:
wd="analyses/1012"


In [12]:
!mkdir $wd

In [14]:
cd $wd

/Users/sr320/Documents/GitHub/nb-2018/O_lurida/analyses/1012


In [18]:
!/Applications/bioinfo/ncbi-blast-2.7.1+/bin/blastx \
-query ../../analyses/dml010.tile.1kslop.fa \
-db /Users/sr320/Documents/GitHub/fun-gen/blastdb/uniprot_sprot_r2018_08 \
-out dml010.tile.1kslop-uniprot_blastx4slim.tab \
-evalue 1E-05 \
-num_threads 4 \
-max_target_seqs 10 \
-outfmt 6

### In this directory you will need three files
1) blastout file in format `-6`    
2) Uniprot GO annotation file (340M) available here `http://owl.fish.washington.edu/halfshell/bu-alanine-wd/17-07-20/uniprot-SP-GO.sorted`    
3) GOslim file available here `http://owl.fish.washington.edu/halfshell/bu-alanine-wd/17-07-20/GO-GOslim.sorted`

In [19]:
!curl -O http://owl.fish.washington.edu/halfshell/bu-alanine-wd/17-07-20/uniprot-SP-GO.sorted

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  340M  100  340M    0     0  48.6M      0  0:00:07  0:00:07 --:--:-- 73.4M      0  0:00:08  0:00:06  0:00:02 45.8M


In [20]:
!curl -O http://owl.fish.washington.edu/halfshell/bu-alanine-wd/17-07-20/GO-GOslim.sorted

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2314k  100 2314k    0     0  2314k      0  0:00:01 --:--:--  0:00:01 11.0M


In [21]:
!head dml010.tile.1kslop-uniprot_blastx4slim.tab

Contig15444:3226-4225	sp|P04323|POL3_DROME	38.667	225	132	2	672	13	201	424	9.21e-46	170
Contig15444:3226-4225	sp|Q99315|YG31B_YEAST	43.158	190	106	1	609	40	610	797	1.37e-43	164
Contig15444:3226-4225	sp|Q7LHG5|YI31B_YEAST	43.158	190	106	1	609	40	636	823	1.47e-43	163
Contig15444:3226-4225	sp|P20825|POL2_DROME	32.950	261	170	1	780	13	163	423	6.77e-43	161
Contig15444:3226-4225	sp|P10394|POL4_DROME	34.263	251	159	1	762	28	277	527	2.32e-41	157
Contig15444:3226-4225	sp|Q8I7P9|POL5_DROME	34.043	235	150	1	702	13	106	340	6.33e-40	153
Contig15444:3226-4225	sp|P10401|POLY_DROME	32.661	248	153	3	726	13	156	399	8.91e-38	146
Contig15444:3226-4225	sp|P15629|POL_SOCMV	34.934	229	140	5	687	25	189	416	7.97e-37	143
Contig15444:3226-4225	sp|P0CT40|TF29_SCHPO	29.391	279	181	4	816	22	345	621	7.74e-36	141
Contig15444:3226-4225	sp|P0CT37|TF24_SCHPO	29.391	279	181	4	816	22	345	621	7.74e-36	141


In [22]:
#checking if files in directory
!ls 

GO-GOslim.sorted
dml010.tile.1kslop-uniprot_blastx4slim.tab
uniprot-SP-GO.sorted


### Set blastout file as variable

In [23]:
blastout="dml010.tile.1kslop-uniprot_blastx4slim.tab"


# That should be the last thing you have to Type!

In [24]:
!head -2 $blastout

Contig15444:3226-4225	sp|P04323|POL3_DROME	38.667	225	132	2	672	13	201	424	9.21e-46	170
Contig15444:3226-4225	sp|Q99315|YG31B_YEAST	43.158	190	106	1	609	40	610	797	1.37e-43	164


In [25]:
#convert pipes to tab
!tr '|' '\t' < $blastout \
> _blast-sep.tab

In [26]:
!head -2 _blast-sep.tab

Contig15444:3226-4225	sp	P04323	POL3_DROME	38.667	225	132	2	672	13	201	424	9.21e-46	170
Contig15444:3226-4225	sp	Q99315	YG31B_YEAST	43.158	190	106	1	609	40	610	797	1.37e-43	164


In [27]:
#reducing number of columns and sorting to get spid evlaue - 
!awk -v OFS='\t' '{print $3, $1, $13}' < _blast-sep.tab | sort \
> _blast-sort.tab

In [28]:
!head -2 _blast-sort.tab

A2AJA7	Contig16849:11076-12075	1.18e-14
A2AJA7	Contig16849:11076-12075	1.95e-21


In [29]:
#joining blast with uniprot annoation file and reducing to three columns UniprotID, Query, All GO terms
!join -t $'\t' \
_blast-sort.tab \
uniprot-SP-GO.sorted \
| cut -f1,2,14 \
> _blast-annot.tab

In [30]:
!head -2 _blast-annot.tab
#should look like
#A1BHN5	CHOYP_AAAS.3.3	GO:0005524; GO:0006298; GO:0030983
#A5PLL7	CHOYP_AAEL_AAEL001208.1.1	GO:0005737; GO:0005783; GO:0005789; GO:0016021; GO:0031625; GO:0061630


A2AJA7	Contig16849:11076-12075	GO:0015031; GO:0016021
A2AJA7	Contig16849:11076-12075	GO:0015031; GO:0016021


## The following is a script modidified from Sam

In [31]:
%%bash 

# This script was originally written to address a specific problem that Rhonda was having



# input_file is the initial, "problem" file
# file is an intermediate file that most of the program works upon
# output_file is the final file produced by the script
input_file="_blast-annot.tab"
file="_intermediate.file"
output_file="_blast-GO-unfolded.tab"

# sed command substitutes the "; " sequence to a tab and writes the new format to a new file.
# This character sequence is how the GO terms are delimited in their field.
sed $'s/; /\t/g' "$input_file" > "$file"

# Identify first field containing a GO term.
# Search file with grep for "GO:" and pipe to awk.
# Awk sets tab as field delimiter (-F'\t'), runs a for loop that looks for "GO:" (~/GO:/), and then prints the field number).
# Awk results are piped to sort, which sorts unique by number (-ug).
# Sort results are piped to head to retrieve the lowest value (i.e. the top of the list; "-n1").
begin_goterms=$(grep "GO:" "$file" | awk -F'\t' '{for (i=1;i<=NF;i++) if($i ~/GO:/) print i}' | sort -ug | head -n1)

# While loop to process each line of the input file.
while read -r line
	do
	
	# Send contents of the current line to awk.
	# Set the field separator as a tab (-F'\t') and print the number of fields in that line.
	# Save the results of the echo/awk pipe (i.e. number of fields) to the variable "max_field".
	max_field=$(echo "$line" | awk -F'\t' '{print NF}')

	# Send contents of current line to cut.
	# Cut fields (i.e. retain those fields) 1-12.
	# Save the results of the echo/cut pipe (i.e. fields 1-12) to the variable "fixed_fields"
	fixed_fields=$(echo "$line" | cut -f1-2)

	# Since not all the lines contain the same number of fields (e.g. may not have GO terms),
	# evaluate the number of fields in each line to determine how to handle current line.

	# If the value in max_field is less than the field number where the GO terms begin,
	# then just print the current line (%s) followed by a newline (\n).
	if (( "$max_field" < "$begin_goterms" ))
		then printf "%s\n" "$line"
			else

			# Send contents of current line (which contains GO terms) to cut.
			# Cut fields (i.e. retain those fields) 13 to whatever the last field is in the curent line.
			# Save the results of the echo/cut pipe (i.e. all the GO terms fields) to the variable "goterms".
			goterms=$(echo "$line" | cut -f"$begin_goterms"-"$max_field")
			
			# Assign values in the variable "goterms" to a new indexed array (called "array"), 
			# with tab delimiter (IFS=$'\t')
			IFS=$'\t' read -r -a array <<<"$goterms"
			
			# Iterate through each element of the array.
			# Print the first 12 fields (i.e. the fields stored in "fixed_fields") followed by a tab (%s\t).
			# Print the current element in the array (i.e. the current GO term) followed by a new line (%s\n).
			for element in "${!array[@]}"	
				do printf "%s\t%s\n" "$fixed_fields" "${array[$element]}"
			done
	fi

# Send the input file into the while loop and send the output to a file named "rhonda_fixed.txt".
done < "$file" > "$output_file"

In [32]:
!head _blast-GO-unfolded.tab
#should look like
#A1BHN5	CHOYP_AAAS.3.3	GO:0005524
#A1BHN5	CHOYP_AAAS.3.3	GO:0006298
#A1BHN5	CHOYP_AAAS.3.3	GO:0030983
#A5PLL7	CHOYP_AAEL_AAEL001208.1.1	GO:0005737
#A5PLL7	CHOYP_AAEL_AAEL001208.1.1	GO:0005783


A2AJA7	Contig16849:11076-12075	GO:0015031
A2AJA7	Contig16849:11076-12075	GO:0016021
A2AJA7	Contig16849:11076-12075	GO:0015031
A2AJA7	Contig16849:11076-12075	GO:0016021
A2AJA7	Contig16849:11076-12075	GO:0015031
A2AJA7	Contig16849:11076-12075	GO:0016021
A2AJA7	Contig16849:11076-12075	GO:0015031
A2AJA7	Contig16849:11076-12075	GO:0016021
A2AJX4	Contig16849:11076-12075	GO:0005737
A2AJX4	Contig16849:11076-12075	GO:0005794


In [62]:
!awk '{print $3,"\t" $2}' _blast-GO-unfolded.tab | gsort -V | sed 's/ //' > _blast-GO-unfolded.sorted
#hack with sed to rmove extra space after GO

In [63]:
!head _blast-GO-unfolded.sorted 


GO:0000082	Contig8544:2726-3725
GO:0000082	Contig8544:2726-3725
GO:0000166	Contig8544:2726-3725
GO:0000166	Contig8544:2726-3725
GO:0000278	Contig8544:2726-3725
GO:0000722	Contig8544:2726-3725
GO:0000731	Contig8544:2726-3725
GO:0000731	Contig8544:2726-3725
GO:0000943	Contig4714:526-1525
GO:0000943	Contig15444:3226-4225


In [77]:
!cat _blast-GO-unfolded.sorted | cut -f1 | sort | uniq -c > forrevigo.txt

In [40]:
!head GO-GOslim.sorted

GO:0000001	mitochondrion inheritance	cell organization and biogenesis	P
GO:0000002	mitochondrial genome maintenance	cell organization and biogenesis	P
GO:0000003	reproduction	other biological processes	P
GO:0000006	high affinity zinc uptake transmembrane transporter activity	transporter activity	F
GO:0000007	low-affinity zinc ion transmembrane transporter activity	transporter activity	F
GO:0000009	"alpha-1,6-mannosyltransferase activity"	other molecular function	F
GO:0000010	trans-hexaprenyltranstransferase activity	other molecular function	F
GO:0000011	vacuole inheritance	cell organization and biogenesis	P
GO:0000012	single strand break repair	DNA metabolism	P
GO:0000012	single strand break repair	stress response	P


In [69]:
!join -1 1 -2 1 -t $'\t' \
_blast-GO-unfolded.sorted \
GO-GOslim.sorted | cat

GO:0000082	Contig8544:2726-3725	G1/S transition of mitotic cell cycle	cell cycle and proliferation	P
GO:0000082	Contig8544:2726-3725	G1/S transition of mitotic cell cycle	cell cycle and proliferation	P
GO:0000166	Contig8544:2726-3725	nucleotide binding	other molecular function	F
GO:0000166	Contig8544:2726-3725	nucleotide binding	other molecular function	F
GO:0000278	Contig8544:2726-3725	mitotic cell cycle	cell cycle and proliferation	P
GO:0000722	Contig8544:2726-3725	telomere maintenance via recombination	DNA metabolism	P
GO:0000722	Contig8544:2726-3725	telomere maintenance via recombination	cell organization and biogenesis	P
GO:0000731	Contig8544:2726-3725	DNA synthesis during DNA repair	DNA metabolism	P
GO:0000731	Contig8544:2726-3725	DNA synthesis during DNA repair	stress response	P
GO:0000731	Contig8544:2726-3725	DNA synthesis during DNA repair	DNA metabolism	P
GO:0000731	Contig8544:2726-3725	DNA synthesis during DNA repair	stress response	P
GO:0000943	Contig4714:526-1525	retrotran

GO:0020037	Contig35213:1226-2225	heme binding	other molecular function	F
GO:0020037	Contig35213:1226-2225	heme binding	other molecular function	F
GO:0020037	Contig35213:1226-2225	heme binding	other molecular function	F
GO:0020037	Contig35213:1226-2225	heme binding	other molecular function	F
GO:0020037	Contig35213:1226-2225	heme binding	other molecular function	F
GO:0020037	Contig35213:1226-2225	heme binding	other molecular function	F
GO:0020037	Contig35213:1226-2225	heme binding	other molecular function	F
GO:0020037	Contig35213:1226-2225	heme binding	other molecular function	F
GO:0020037	Contig35213:1226-2225	heme binding	other molecular function	F
GO:0020037	Contig35213:1226-2225	heme binding	other molecular function	F
GO:0020037	Contig35213:1226-2225	heme binding	other molecular function	F
GO:0020037	Contig35213:1226-2225	heme binding	other molecular function	F
GO:0020037	Contig35213:1226-2225	heme binding	other molecular function	F
GO:0020037	Contig35213:1226-2225	heme binding	other

In [75]:
!join -1 1 -2 1 -t $'\t' \
_blast-GO-unfolded.sorted \
GO-GOslim.sorted | cut -f1 | sort | uniq -c

   2 GO:0000082
   2 GO:0000166
   1 GO:0000278
   2 GO:0000722
   4 GO:0000731
   5 GO:0000943
   3 GO:0001764
   9 GO:0003676
  20 GO:0003677
   2 GO:0003682
  18 GO:0003723
  20 GO:0003887
  33 GO:0003964
 188 GO:0004129
  31 GO:0004190
  20 GO:0004519
  13 GO:0004523
   7 GO:0004533
   5 GO:0004540
   7 GO:0005198
 188 GO:0005506
   4 GO:0005507
   5 GO:0005524
  65 GO:0005576
   1 GO:0005615
   9 GO:0005634
   2 GO:0005654
  15 GO:0005737
 338 GO:0005743
  38 GO:0005751
   3 GO:0005768
  17 GO:0005794
   1 GO:0005856
  12 GO:0005886
 169 GO:0006119
  19 GO:0006123
   2 GO:0006260
   1 GO:0006270
   2 GO:0006272
   4 GO:0006287
   4 GO:0006297
  18 GO:0006310
   1 GO:0006313
   1 GO:0006338
   1 GO:0007420
   6 GO:0008233
  14 GO:0008270
   6 GO:0008289
   2 GO:0008310
   2 GO:0008622
 188 GO:0009060
  11 GO:0015031
  26 GO:0015074
  19 GO:0015990
  66 GO:0016020
 216 GO:0016021
   7 GO:0019013
   7 GO:0019076
   7 GO:0020002
 188 GO:0020037
   1 GO:0021527
   1 GO:0021799
   4 GO:

In [65]:
#joining files to get GOslim for each query (with duplicate GOslim / query removed)
!join -1 1 -2 1 -t $'\t' \
_blast-GO-unfolded.sorted \
GO-GOslim.sorted \
| uniq | awk -F'\t' -v OFS='\t' '{print $2, $4, $5}' \
> Blastquery-GOslim.tab
!wc -l Blastquery-GOslim.tab

     591 Blastquery-GOslim.tab


In [66]:
!head Blastquery-GOslim.tab
!wc -l Blastquery-GOslim.tab

Contig8544:2726-3725	cell cycle and proliferation	P
Contig8544:2726-3725	other molecular function	F
Contig8544:2726-3725	cell cycle and proliferation	P
Contig8544:2726-3725	DNA metabolism	P
Contig8544:2726-3725	cell organization and biogenesis	P
Contig8544:2726-3725	DNA metabolism	P
Contig8544:2726-3725	stress response	P
Contig8544:2726-3725	DNA metabolism	P
Contig8544:2726-3725	stress response	P
Contig4714:526-1525	nucleus	C
     591 Blastquery-GOslim.tab


# Get P - query and slim

In [68]:
!awk -F"\t" '$3 == "P" { print $1"\t"$2 }' Blastquery-GOslim.tab | sort > Blastquery-GOslim.sorted
!cat Blastquery-GOslim.sorted


Contig15444:3226-4225	DNA metabolism
Contig15444:3226-4225	DNA metabolism
Contig15444:3226-4225	DNA metabolism
Contig15444:3226-4225	other biological processes
Contig16849:11076-12075	cell organization and biogenesis
Contig16849:11076-12075	cell-cell signaling
Contig16849:11076-12075	developmental processes
Contig16849:11076-12075	developmental processes
Contig16849:11076-12075	developmental processes
Contig16849:11076-12075	developmental processes
Contig16849:11076-12075	developmental processes
Contig16849:11076-12075	other biological processes
Contig16849:11076-12075	transport
Contig174856:0-825	DNA metabolism
Contig174856:0-825	DNA metabolism
Contig174856:0-825	other biological processes
Contig174856:0-825	other biological processes
Contig237394:6076-7022	DNA metabolism
Contig237394:6076-7022	DNA metabolism
Contig237394:6076-7022	other biological processes
Contig27772:4126-5125	cell organization and biogenesis
Contig35213:1226-2225	other metabolic processes
Contig35213:1226-2225	oth